# Bài 4: Phân loại dữ liệu theo K-Fold Cross Validation

## Yêu cầu:
- Hiểu và áp dụng K-Fold Cross Validation
- Stratified K-Fold cho dữ liệu không cân bằng
- Đánh giá mô hình với K-Fold

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, cross_validate
from sklearn.datasets import load_iris, make_classification
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from collections import Counter

## 1. K-Fold Cross Validation cơ bản

In [ ]:
# Load dữ liệu
iris = load_iris()
X = iris.data
y = iris.target

print(f"Tổng số mẫu: {len(X)}")
print(f"Phân bố các lớp: {Counter(y)}")

### 1.1. Minh họa K-Fold (K=5)

In [ ]:
# Tạo K-Fold với K=5
kf = KFold(n_splits=5, shuffle=True, random_state=42)

print("\nCách chia K-Fold (K=5):\n")
for fold, (train_idx, test_idx) in enumerate(kf.split(X), 1):
    print(f"Fold {fold}:")
    print(f"  Train: {len(train_idx)} mẫu (indices: {train_idx[:5]}...{train_idx[-5:]})")
    print(f"  Test:  {len(test_idx)} mẫu (indices: {test_idx})")
    print(f"  Phân bố train: {Counter(y[train_idx])}")
    print(f"  Phân bố test:  {Counter(y[test_idx])}")
    print()

### 1.2. Visualize K-Fold

In [ ]:
def visualize_kfold(X, kf, title="K-Fold Visualization"):
    """
    Visualize K-Fold splits
    """
    n_samples = len(X)
    n_splits = kf.get_n_splits()
    
    fig, ax = plt.subplots(figsize=(12, n_splits))
    
    for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
        # Tạo mask
        indices = np.zeros(n_samples)
        indices[test_idx] = 1
        
        # Vẽ
        ax.scatter(range(n_samples), [fold] * n_samples, 
                   c=indices, cmap='RdYlBu', 
                   marker='_', lw=10, vmin=0, vmax=1)
    
    ax.set_xlabel('Sample index', fontsize=12)
    ax.set_ylabel('Fold', fontsize=12)
    ax.set_title(title, fontsize=14)
    ax.set_yticks(range(n_splits))
    ax.set_yticklabels([f'Fold {i+1}' for i in range(n_splits)])
    
    # Legend
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='blue', label='Train'),
        Patch(facecolor='red', label='Test')
    ]
    ax.legend(handles=legend_elements, loc='upper right')
    
    plt.tight_layout()
    plt.show()

# Visualize
kf = KFold(n_splits=5, shuffle=True, random_state=42)
visualize_kfold(X, kf, "5-Fold Cross Validation")

## 2. Stratified K-Fold

In [ ]:
# Tạo dữ liệu không cân bằng
X_imb, y_imb = make_classification(
    n_samples=200,
    n_features=20,
    n_informative=15,
    n_classes=3,
    weights=[0.6, 0.3, 0.1],
    random_state=42
)

print(f"\nDữ liệu không cân bằng:")
print(f"Phân bố: {Counter(y_imb)}")
print(f"Tỷ lệ: {[f'{v/len(y_imb)*100:.1f}%' for v in Counter(y_imb).values()]}")

### 2.1. So sánh K-Fold và Stratified K-Fold

In [ ]:
# K-Fold thường
kf = KFold(n_splits=5, shuffle=True, random_state=42)
print("\nK-Fold thường:")
for fold, (train_idx, test_idx) in enumerate(kf.split(X_imb), 1):
    train_dist = Counter(y_imb[train_idx])
    test_dist = Counter(y_imb[test_idx])
    print(f"Fold {fold}: Train {dict(train_dist)}, Test {dict(test_dist)}")

# Stratified K-Fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
print("\nStratified K-Fold:")
for fold, (train_idx, test_idx) in enumerate(skf.split(X_imb, y_imb), 1):
    train_dist = Counter(y_imb[train_idx])
    test_dist = Counter(y_imb[test_idx])
    print(f"Fold {fold}: Train {dict(train_dist)}, Test {dict(test_dist)}")

## 3. Đánh giá mô hình với K-Fold

### 3.1. Sử dụng cross_val_score

In [ ]:
# Tạo mô hình
svm_model = SVC(kernel='rbf', random_state=42)
lr_model = LogisticRegression(max_iter=1000, random_state=42)

# Đánh giá với 5-Fold CV
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# SVM
svm_scores = cross_val_score(svm_model, X, y, cv=cv, scoring='accuracy')
print("\nSVM với 5-Fold CV:")
print(f"Accuracy mỗi fold: {svm_scores}")
print(f"Mean Accuracy: {svm_scores.mean():.4f} (+/- {svm_scores.std() * 2:.4f})")

# Logistic Regression
lr_scores = cross_val_score(lr_model, X, y, cv=cv, scoring='accuracy')
print("\nLogistic Regression với 5-Fold CV:")
print(f"Accuracy mỗi fold: {lr_scores}")
print(f"Mean Accuracy: {lr_scores.mean():.4f} (+/- {lr_scores.std() * 2:.4f})")

### 3.2. Sử dụng cross_validate (nhiều metrics)

In [ ]:
# Định nghĩa nhiều metrics
scoring = {
    'accuracy': 'accuracy',
    'precision_macro': 'precision_macro',
    'recall_macro': 'recall_macro',
    'f1_macro': 'f1_macro'
}

# Đánh giá SVM
svm_results = cross_validate(svm_model, X, y, cv=cv, scoring=scoring, return_train_score=True)

print("\nKết quả SVM với nhiều metrics:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    test_scores = svm_results[f'test_{metric}']
    print(f"{metric.capitalize():15s}: {test_scores.mean():.4f} (+/- {test_scores.std() * 2:.4f})")

# Đánh giá Logistic Regression
lr_results = cross_validate(lr_model, X, y, cv=cv, scoring=scoring, return_train_score=True)

print("\nKết quả Logistic Regression với nhiều metrics:")
for metric in ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']:
    test_scores = lr_results[f'test_{metric}']
    print(f"{metric.capitalize():15s}: {test_scores.mean():.4f} (+/- {test_scores.std() * 2:.4f})")

## 4. Manual K-Fold Implementation

In [ ]:
def manual_kfold_evaluation(model, X, y, k=5):
    """
    Thực hiện K-Fold CV thủ công và tính các metrics
    """
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
    
    results = {
        'accuracy': [],
        'precision': [],
        'recall': [],
        'f1': []
    }
    
    print(f"\nThực hiện {k}-Fold Cross Validation thủ công:\n")
    
    for fold, (train_idx, test_idx) in enumerate(kf.split(X, y), 1):
        # Chia dữ liệu
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        # Train
        model.fit(X_train, y_train)
        
        # Predict
        y_pred = model.predict(X_test)
        
        # Tính metrics
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred, average='macro')
        rec = recall_score(y_test, y_pred, average='macro')
        f1 = f1_score(y_test, y_pred, average='macro')
        
        results['accuracy'].append(acc)
        results['precision'].append(prec)
        results['recall'].append(rec)
        results['f1'].append(f1)
        
        print(f"Fold {fold}:")
        print(f"  Accuracy:  {acc:.4f}")
        print(f"  Precision: {prec:.4f}")
        print(f"  Recall:    {rec:.4f}")
        print(f"  F1-Score:  {f1:.4f}")
        print()
    
    # Tính trung bình
    print("\nKết quả trung bình:")
    for metric, scores in results.items():
        scores = np.array(scores)
        print(f"{metric.capitalize():10s}: {scores.mean():.4f} (+/- {scores.std() * 2:.4f})")
    
    return results

# Test
svm_model = SVC(kernel='rbf', random_state=42)
results = manual_kfold_evaluation(svm_model, X, y, k=5)

## 5. Visualize kết quả K-Fold

In [ ]:
# Vẽ biểu đồ so sánh
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
fig.suptitle('K-Fold Cross Validation Results', fontsize=16)

metrics = ['accuracy', 'precision', 'recall', 'f1']
positions = [(0, 0), (0, 1), (1, 0), (1, 1)]

for metric, pos in zip(metrics, positions):
    ax = axes[pos]
    scores = results[metric]
    
    # Box plot
    ax.boxplot([scores], labels=['SVM'])
    ax.scatter([1] * len(scores), scores, alpha=0.5, c='red')
    ax.set_ylabel('Score')
    ax.set_title(f'{metric.capitalize()}')
    ax.grid(True, alpha=0.3)
    
    # Thêm mean line
    ax.axhline(y=np.mean(scores), color='green', linestyle='--', 
               label=f'Mean: {np.mean(scores):.4f}')
    ax.legend()

plt.tight_layout()
plt.show()

## Bài tập thực hành

1. Thử nghiệm với các giá trị K khác nhau (3, 5, 10) và so sánh kết quả
2. So sánh K-Fold và Stratified K-Fold trên dữ liệu không cân bằng
3. Đánh giá nhiều mô hình khác nhau (SVM, Logistic Regression, Decision Tree) với K-Fold
4. Giải thích khi nào nên dùng Stratified K-Fold
5. Tại sao K-Fold tốt hơn train/test split đơn giản?